<a href="https://colab.research.google.com/github/skdding3/CSAI_assignment/blob/main/N422a_Distributed_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [ ]:
!pip install gensim --upgrade

     |████████████████████████████████| 23.9 MB 91 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim

gensim.__version__

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


'4.0.1'

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [ ]:
from google.colab import files

file = files.upload()

Saving spam.csv to spam.csv


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('/content/spam.csv', delimiter=',', encoding="latin-1")

df = df[['v1', 'v2']]
features = df['v2']
label = df['v1']

df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
encoder = LabelEncoder()
label_encoded = encoder.fit_transform(label)
label_encoded

array([0, 0, 1, ..., 0, 0, 0])

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [ ]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(features)

X_encoded = tokenizer.texts_to_sequences(features)

max_len=max(len(sent) for sent in X_encoded)
print(max_len)

172


In [ ]:
X_train=pad_sequences(X_train_encoded, maxlen=150, padding='post')
y_train=np.array(y_train)

X_test=pad_sequences(X_test_encoded, maxlen=150, padding='post')
y_test=np.array(y_test)

In [ ]:
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, label_encoded, test_size = 0.15, random_state = 42)
print(f"Train set shape : {X_train.shape}")
print(f"Test set shape : {X_test.shape}")

Train set shape : (4736, 150)
Test set shape : (836, 150)


In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))

def get_vector(word):
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
60/60 [==============================] - 1s 8ms/step - loss: 0.6766 - acc: 0.8229 - val_loss: 0.6567 - val_acc: 0.8660
Epoch 2/10
60/60 [==============================] - 0s 6ms/step - loss: 0.6405 - acc: 0.8648 - val_loss: 0.6226 - val_acc: 0.8660
Epoch 3/10
60/60 [==============================] - 0s 6ms/step - loss: 0.6089 - acc: 0.8648 - val_loss: 0.5932 - val_acc: 0.8660
Epoch 4/10
60/60 [==============================] - 0s 5ms/step - loss: 0.5819 - acc: 0.8648 - val_loss: 0.5673 - val_acc: 0.8660
Epoch 5/10
60/60 [==============================] - 0s 6ms/step - loss: 0.5585 - acc: 0.8648 - val_loss: 0.5455 - val_acc: 0.8660
Epoch 6/10
60/60 [==============================] - 0s 6ms/step - loss: 0.5383 - acc: 0.8648 - val_loss: 0.5265 - val_acc: 0.8660
Epoch 7/10
60/60 [==============================] - 0s 6ms/step - loss: 0.5208 - acc: 0.8648 - val_loss: 0.5098 - val_acc: 0.8660
Epoch 8/10
60/60 [==============================] - 0s 6ms/step - loss: 0.5057 - acc: 0.86

lose 값 0.4810

acc 값 0.8648